# <center> Get user graph for `Russian`

In [10]:
import sys
import util
import json
import time
from twython import Twython

### Twitter API credentials

In [18]:
APP_KEY = "AV4KujYNS1y6HdmQ4zXxHw2Tx"
APP_SECRET = "xd2FuFaS5zE1ON407C3U5X0kdEtRJIjH9OMDmgyMmxQCZMFlli"
OAUTH_TOKEN = "1058011325316636672-HkpzhyiOHdJkuwi3FD9QrkMFbuHuVt"
OAUTH_TOKEN_SECRET = "iiD75H9ERtkATnd74pVRBfM7TI189BSmDasYzN2uUO123"

### Retrieve user graph

Get user set.

In [3]:
users = list()
for line in open("../Datasets/russian_election2018_rtu.txt"):
    line = line.split()
    users.append(int(line[2]))
    users.append(int(line[3]))
# users = util.get_authors("../Datasets/russian_rtid.txt")

Get the followgraph of users present in the trace.

In [ ]:
# init
twitter = Twython(APP_KEY, APP_SECRET, OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
FollowGraph = dict()
outfile = "../Datasets/russian_adjList.txt"

for i,u in enumerate(users):
    
    # reinit cursor
    cursor = -1

    # while there are followers left for this user
    while cursor != 0:
        sys.stdout.write("User {} / {}...\r".format(i, len(users)))

        try:
            # get followers
            result = twitter.get_followers_ids(id=u, cursor=cursor)
            ids = result['ids']
            cursor = result['next_cursor']

            # write to out
            with open(outfile, 'a') as out:
                for v in ids:
                    out.write("{} {}\n".format(u,v))

            # wait if API rate limit has been reached
            calls_remaining = int(twitter.get_lastfunction_header('x-rate-limit-remaining'))
            if calls_remaining == 0:
                time_to_wait = 15*60
                start = time.time()
                elapsed_time = 0
                while elapsed_time < time_to_wait:
                    sys.stdout.flush()
                    sys.stdout.write("User {} / {}... Rate limit reached... {} seconds to wait.\r".format(i, len(users), time_to_wait - elapsed_time))
                    time.sleep(1)
                    elapsed_time = time.time() - start
                    
        except:
            cursor += 1
            continue
            
                    
#         except:
#             cursor += 1
#             continue
                    
                
#             # twitter api rate limit exceeded --> sleep then reconnect and retry
#             except:
#                 waiting_time = float(twitter.get_lastfunction_header('x-rate-limit-reset'))
# #                 del twitter
#                 start = time.time()
#                 elapsed_time = time.time() - start
#                 while elapsed_time <= waiting_time:
#                     sys.stdout.flush()
#                     sys.stdout.write("Elapsed time : {} on {} total time to wait.\n".format(elapsed_time, waiting_time))
# #                     sys.stdout.write("Sleeping... {:.0f}min {:.0f}sec left\r".format(time_left//60, time_left%60))
#                     time.sleep(1)
#                     elapsed_time = time.time() - start
# #                 twitter = Twython(APP_KEY, APP_SECRET, OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
#                 retry = True

In [11]:
int(twitter.get_lastfunction_header('x-rate-limit-reset'))

1563219410

In [8]:
twitter = Twython(APP_KEY, APP_SECRET, OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
twitter.show_status(id=1150759441941368832)
twitter.get_lastfunction_header('x-rate-limit-remaining')

'898'

## VERIF

Get followgraph from .txt file.

In [1]:
followgraph = dict()
for line in open("../Datasets/russian_adjList.txt"):
    lead, follow = int(line.split()[0]), int(line.split()[1])
    if lead in followgraph:
        followgraph[lead].add(follow)
    else:
        followgraph[lead] = {follow}

In [5]:
len(followgraph[832067407])

22451

problematic users

In [6]:
users = set()
for line in open("../Datasets/russian_getfollowers_error_logs.txt"):
    line = line.split()
    if int(line[1]) != -1:
        users.add(int(line[0]))

In [11]:
len(users)

21

re-crawl problematic users

In [19]:
followgraph = dict()
twitter = Twython(APP_KEY, APP_SECRET, OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
for i,u in enumerate(users):
    
    followgraph[u] = set()
    
    # reinit cursor
    cursor = -1
    
    # while there are followers left for this user
    while cursor != 0:
        
        try:
            # get followers
            result = twitter.get_followers_ids(id=u, cursor=cursor)
            sys.stdout.flush()
            sys.stdout.write("User {} / {} ok...\r".format(i+1, len(users)))


            # write to out
            for v in result['ids']:
                followgraph[user].add(v)
            cursor = result['next_cursor']

            # wait if API rate limit has been reached
            calls_remaining = int(twitter.get_lastfunction_header('x-rate-limit-remaining'))
            if calls_remaining == 0:
                time_to_wait = 15*60 + 5 # add 5 seconds just in case
                start = time.time()
                elapsed_time = 0
                while elapsed_time < time_to_wait:
                    sys.stdout.flush()
                    sys.stdout.write("User {} / {}... Rate limit reached... {} seconds to wait.\r".format(i, len(users), round(time_to_wait - elapsed_time)))
                    time.sleep(1)
                    elapsed_time = time.time() - start
                    
        except:
            sys.stdout.flush()
            sys.stdout.write("User {} / {} ERROR...\r".format(i, len(users)))
            cursor += 1
            continue

In [23]:
for u in users:
    print(u)
    print(twitter.show_user(id=u)['followers_count'])
    print(len(followgraph[u]))
    print()

938465725932621824
0
0

870884964618960896
70
0

921241774953287680
630
0

147655301
24015
0

423660053
17
0

4895431835
53
0

28859692
223984
0

2742997421
2160
0

3293820729
150
0

171309893
23036
0

300747334
17555
0

387840841
522926
0

542310868
78
0

3024839255
12
0

194023516
329
0

2592819421
3
0

2832071262
33395
0

43429477
89589
0

2955196012
87
0

3080357232
140
0

2907348474
5276
0



In [24]:
result = twitter.get_followers_ids(id=423660053)

TwythonRateLimitError: Twitter API returned a 429 (Too Many Requests), Rate limit exceeded